Reminder: IO‑bound operations are operations associated with long waiting for another device, for example, a network card or a disk.

<center>
<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*Tr_9lhhvY2Kpga-1V5jNCg.png" alt="io-operations" width=800/>
</center>


### DB operation

In [ ]:
from datetime import date
import pymongo

client = MongoClient('localhost', 27017)

posts = client['web_db']['posts']

# blocks until DB answers
results = posts.find({'author': 'Somebody', 'date': date.today})

### Http request

In [ ]:
import requests

# blocks until site returns response
response = requests.get('http://some.site')

## Asynchronous I/O

Let's assume we have some IO-bound tasks, then:

![image](./async_vs_sync.png)

Asynchronous I/O — non‑blocking input/output processing that allows execution to continue without waiting for the data transfer to finish.

# Coroutines (coroutines)

### from the very beginning...
...

In [ ]:
def eager_range(up_to: int) -> list[int]:
    sequence = []
    index = 0
    while index < up_to:
        sequence.append(index)
        index += 1
    return sequence

Starting with Python 2.2, generators appeared in the language

In [ ]:
from collections.abc import Generator

In [ ]:
def lazy_range(up_to: int) -> Generator[int, None, None]:
    index = 0
    while index < up_to:
        yield index
        index += 1

In Python 2.5, the `send()` method was introduced

In [ ]:
def jumping_range(up_to: int) -> Generator[int, int | None, None]:
    index = 0
    while index < up_to:
        jump = yield index
        if jump is None:
            jump = 1
        index += jump

In [ ]:
generator = jumping_range(5)

In [ ]:
print('next   :', next(generator))  # by default, `yield index` returns None
print('send  2:', generator.send(2))  # basically, next(generator) but `yield index` returns 2
print('next   :', next(generator))
print('send -1:', generator.send(-1))

In Python 3.3, the important syntactic sugar `yield from` was added

`yield from` is equivalent to:
```python
for x in generator_fn():
    yield x

(for x in generator_fn())

yield from generator_fn()
```

In [ ]:
def bottom() -> Generator[int, None, int]:
    yield 42
    yield 84
    return 0

In [ ]:
def top() -> Generator[int, None, None]:
    value = yield from bottom()
    print(value)
    return value

In [ ]:
list(top())

In [ ]:
generator = top()
next(generator), next(generator)

In [ ]:
try:
    next(generator)
except StopIteration as e:
    print(e.value)


Finally, in Python 3.4 the `asyncio` framework was introduced

In [ ]:
import asyncio
import types

In [ ]:
@types.coroutine  # asyncio.coroutine
def countdown(n: int) -> Generator[asyncio.Future, None, None]:
    while n > 0:
        print(n)
        yield from asyncio.sleep(1)  # asyncio.sleep might be a coroutine that has several yield points; that's why we need `yield from`
        n -= 1

In [ ]:
next(countdown(3))

In [ ]:
countdown_coro = countdown(3)
next(countdown_coro), next(countdown_coro)

In [ ]:
import asyncio
import types

@types.coroutine
def countdown(n: int):
    while n > 0:
        print(f"Count: {n}")
        yield from asyncio.sleep(1)
        n -= 1
        
# Create coroutine
coro = countdown(3)

# First next() - works fine
print("First next():")
result1 = next(coro)
print(f"Yielded: {result1}")  # This is a Future object
print(f"Future done? {result1.done()}")  # False - it's still pending

# Second next() immediately - ERROR
print("\nSecond next() immediately:")
try:
    result2 = next(coro)
except RuntimeError as e:
    print(f"Error: {e}")

In short: you shouldn't forcefully resume a coroutine. That's a job of event loop.

In [ ]:
# to start nested event loop inside jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [ ]:
loop = asyncio.get_event_loop()
loop.run_until_complete(countdown(3))

It quickly becomes clear that the language now has some confusion between generators and coroutines

And in Python 3.5, `async/await` was introduced, replacing generator‑based coroutines with ones built into the language

In [ ]:
async def compute(a: int, b: int) -> int:
    print('Compute...')
    await asyncio.sleep(1)
    return a + b

In [ ]:
compute(3, 5)

In [ ]:
next(compute(3, 5))

In [ ]:
next(compute(3, 5).__await__())  # awaitable

In [ ]:
loop = asyncio.get_event_loop()
loop.run_until_complete(compute(3, 5))

optional

And then in Python 3.6, asynchronous generators became possible

In [ ]:
async def ticker(upto: int) -> Generator[int, None, None]:
    for i in range(upto):
        await asyncio.sleep(1)
        yield i

In [ ]:
next(ticker(2).__anext__().__await__())

In [ ]:
loop = asyncio.get_event_loop()
loop.run_until_complete(ticker(2).__anext__())  # anext(...) from 3.10

# Event Loop

You can think of coroutine as a sophisticated generator and of event loop as a sophisticated for loop over your coroutines.

`async` is a coroutine marker, and `await` is a coroutine's version of `yield from`.

In [ ]:
async def compute(a: int, b: int) -> int:
    print('Compute...')
    await asyncio.sleep(1)
    return a + b

In [ ]:
async def print_sum(a: int, b: int) -> None:
    result = await compute(a, b)
    print(f'{a} + {b} = {result}')

In [ ]:
loop = asyncio.get_event_loop()
loop.run_until_complete(print_sum(1, 2))

<center>
<img src="http://ntoll.org/static/images/tulip_coro.png" alt="event-loop" width=1200/>
</center>

In [ ]:
# Simplified code from
# https://github.com/python/cpython/blob/1d1bb95abdcafe92c771fb3dc4722351b032cc24/Lib/asyncio/tasks.py#L630
async def sleep(delay: float) -> None:
    if delay <= 0:
        await __sleep0()
        return result

    loop = asyncio.get_running_loop()
    future = loop.create_future()
    loop.call_later(
        delay,
        future.set_result,
        None,
    )
    
    await future

In [ ]:
# note: sleep0 is used to let take a pause and let other coroutines run

@types.coroutine
def __sleep0() -> Generator[None, None, None]:
    yield

# Modern asyncio

## python 3.9+

## Hello world

In [ ]:
async def main() -> None:
    print('Hello ...')
    await asyncio.sleep(1)
    print('... World!')

In [ ]:
asyncio.run(main())

## Scheduling coroutines

In [ ]:
async def say_after(delay: int, what: str) -> None:
    print(f'wait for {delay}s')
    await asyncio.sleep(delay)
    print(what)

In [ ]:
import time

In [ ]:
print(f"started at {time.strftime('%X')}")

await say_after(1, 'hello')
await say_after(2, 'world')

print(f"finished at {time.strftime('%X')}")

In [ ]:
task1 = asyncio.create_task(say_after(2, 'hello'))
task2 = asyncio.create_task(say_after(1, 'world'))

print(f"started at {time.strftime('%X')}")

await task1
await task2

print(f"finished at {time.strftime('%X')}")

Tasks are used to schedule coroutines to run “concurrently”

When a task is created from a coroutine via `asyncio.create_task()`,

it is automatically scheduled to run on the next event loop tick (run soon)

## asyncio.gather

In [ ]:
async def factorial(name: str, number: int) -> None:
    result = 1
    for i in range(2, number + 1):
        print(f'Task {name}: Compute factorial({i})...')
        await asyncio.sleep(1)
        result *= i
    print(f'Task {name}: factorial({number}) = {result}')
    return result

In [ ]:
await asyncio.gather(
    factorial('A', 2),
    factorial('B', 3),
    factorial('C', 4),
#     return_exceptions=True,
)

## asyncio.TaskGroup
#### python 3.11+

In [ ]:
async with asyncio.TaskGroup() as tg:
    tg.create_task(factorial('A', 2))
    tg.create_task(factorial('B', 3))
    tg.create_task(factorial('C', 4))
    print('tasks pending to start')
print('tasks complete')

## Waiting & timeouts

In [ ]:
async def eternity() -> None:
    await asyncio.sleep(3600)  # 1 hour sleep
    print('yay!')

In [ ]:
try:
    await asyncio.wait_for(eternity(), timeout=1)  # wait for at most 1 second
except TimeoutError:
    print('timeout!')

## Cancelling coroutines

In [ ]:
async def another_eternity() -> None:
    try:
        await asyncio.sleep(3600)
        print(f'{time.strftime("%X")} yay!')
    finally:
        print(f'{time.strftime("%X")} cancelled')
        await asyncio.sleep(5)

In [ ]:
print(f'{time.strftime("%X")} started')
try:
    # Send CancelledError to coro after 1 sec and wait for coro completion
    await asyncio.wait_for(another_eternity(), timeout=1)
except TimeoutError:
    print(f'{time.strftime("%X")} timeout!')
print(f'{time.strftime("%X")} finished')

## Shielding coroutines from cancellation

In [ ]:
async def important_task() -> None:
    print(f'{time.strftime("%X")} important task started')
    await asyncio.sleep(5)
    print(f'{time.strftime("%X")} important task finished')

In [ ]:
print(f'{time.strftime("%X")} started')
try:
    await asyncio.wait_for(asyncio.shield(important_task()), timeout=1.0)
except asyncio.TimeoutError:
    print(f'{time.strftime("%X")} timeout!')
print(f'{time.strftime("%X")} finished')

## as_completed

In [ ]:
async def factorial(number: int) -> tuple[int, int]:
    result = 1
    for i in range(2, number + 1):
        await asyncio.sleep(1)
        result *= i
    return number, result

In [ ]:
for i, future in enumerate(asyncio.as_completed([factorial(4), factorial(3),
                                                 factorial(5), factorial(2)])):
    number, result = await future
    print(f"Factorial({number}) = {result}")

## async with

An asynchronous context manager is a context manager,

that can suspend execution in its enter and exit methods:

`__aenter__()`, `__aexit__()`
...
An asynchronous iterable object must return

an asynchronous iterator from its iter method (`__aiter__()`).

In [ ]:
lock = asyncio.Lock()

async with lock:
    # access shared state

### aiohttp

In [ ]:
import aiohttp

In [ ]:
async with aiohttp.ClientSession() as session:
    async with session.get('http://google.com') as resp:
        text = await resp.text()
        print(text[:70], '...')

In [ ]:
import requests

In [ ]:
with requests.Session() as session:
    with session.get('http://google.com') as resp:
        text = resp.text
        print(text[:70], '...')

## async for

An asynchronous iterable object must return

an asynchronous iterator from its iter method (`__aiter__()`).

An asynchronous iterator must return an awaitable,

which yields the next value of the iterator when awaited,

or raise `StopAsyncIteration` from its next method (`__anext__()`).

In [ ]:
from collections.abc import Iterator

In [ ]:
async def ticker(upto: int) -> Iterator[int]:
    for i in range(upto):
        await asyncio.sleep(1)
        yield i

In [ ]:
async for i in ticker(3):
    print(i, end=' ')

In [ ]:
async_iter = aiter(ticker(2))  # 3.10+
async_iter

In [ ]:
await anext(async_iter)  # 3.10+

In [ ]:
await anext(async_iter)

In [ ]:
await anext(async_iter)

### Motor

In [ ]:
from motor.motor_asyncio import AsyncIOMotorClient

In [ ]:
client = AsyncIOMotorClient('localhost', 27017)

In [ ]:
await client.test_database.test_collection.insert_one({'key': 'value'})

In [ ]:
async for document in client.test_database.test_collection.find({'key': 'value'}):
    print(document)

## Running synchronous code

In [ ]:
def blocking_io() -> None:
    print(f"{time.strftime('%X')} start blocking IO")
    time.sleep(5)
    print(f"{time.strftime('%X')} finished blocking IO")

In [ ]:
non_blocking_io = asyncio.to_thread(blocking_io)

In [ ]:
print(f"{time.strftime('%X')} start gather")
await asyncio.gather(non_blocking_io, asyncio.sleep(5))
print(f"{time.strftime('%X')} finished gather")

In [ ]:
import asyncio
import time

def blocking_io() -> None:
    print(f"{time.strftime('%X')} start blocking IO")
    time.sleep(1)
    print(f"{time.strftime('%X')} finished blocking IO")

async def main():
    # WITHOUT asyncio.to_thread() - Sequential execution
    print("=== WITHOUT asyncio.to_thread() - Sequential ===")
    start = time.time()
    
    blocking_io()
    blocking_io()
    blocking_io()
    
    print(f"Total time: {time.time() - start:.1f} seconds\n")
    
    # WITH asyncio.to_thread() - Parallel execution
    print("=== WITH asyncio.to_thread() - Parallel ===")
    start = time.time()
    
    await asyncio.gather(
        asyncio.to_thread(blocking_io),
        asyncio.to_thread(blocking_io),
        asyncio.to_thread(blocking_io)
    )
    
    print(f"Total time: {time.time() - start:.1f} seconds")

asyncio.run(main())

Good question: How is it possible to run blocking IO in asyncio if we have GIL?

Answer: IO-bound operations release GIL to let other threads run.

## Debugging asyncio

`$ PYTHONASYNCIODEBUG=1 python asyncio_program_to_debug.py`

https://docs.python.org/3/library/asyncio-dev.html